In [1]:
import gnupg
import os
import glob
import shutil
from datetime import datetime
from pathlib import Path
from ftplib import FTP
DEBUG = True

# Head
rootpath = 'C:\\Users\\prashant\\Desktop\\PGP\\FTSol'
# Child
keypath = rootpath + '\\gnupg'
imppath = rootpath + '\\import_key'
exppath = rootpath + '\\export_key'
mediapath = rootpath + '\\media'
# Grand Child
pvtpath = keypath + '\\private-keys-v1.d'
pubpath = keypath + '\\openpgp-revocs.d'
pubkey = exppath + '\\pub_key.asc'
pvtkey = exppath + '\\pvt_key.asc'


In [ ]:
# refresh media directory
shutil.rmtree(mediapath)
os.makedirs(mediapath)


In [2]:
# PGP Settings
EmailKey = "jones.thayil@gmail.com"
KeyPhrase = 'Pass@123'
KeyType = "RSA"
KeyLen = "2048"
UserKey = "FTSPL Server"
Comment = "RSA 2048"
keyext = ('.asc',)


In [3]:
# Auto-Key Import Program
if DEBUG:
    shutil.rmtree(keypath)  # Deleting gnupg
    os.makedirs(keypath)  # Creating gnupg
    os.makedirs(pvtpath)  # Creating gnupg/private-keys-v1.d
    os.makedirs(pubpath)  # Creating gnupg/openpgp-revocs.d
gpg = gnupg.GPG(gnupghome=keypath, verbose=False)
gpg.encoding = 'utf-8'

# Getting client fingerprint if present
keycount = len(glob.glob1(imppath, "*.asc"))
if keycount == 1:
    for num, dirfiles in enumerate(os.listdir(imppath), start=0):
        if dirfiles.endswith(".asc"):
            with open(imppath + '\\' + dirfiles, 'r') as file:
                f = file.read()
                imported_key = gpg.import_keys(f)
                C_fp = gpg.list_keys()[num]['fingerprint']
                gpg.trust_keys(gpg.list_keys()[num]['fingerprint'], 'TRUST_FULLY')
                if DEBUG:
                    print('Index \n', num, '\n')
                    print('key list \n', gpg.list_keys()[num], '\n')
                    print('C_fp \n', gpg.list_keys()[num]['fingerprint'], '\n')

# Generate server fingerprint if not present
for num, dirfiles in enumerate(os.listdir(exppath), start=0):
    if dirfiles.endswith(keyext):
        with open(exppath + '\\pgp_public_key.asc', 'r') as file:
            f = file.read()
            imported_key = gpg.import_keys(f)
            S_fp = gpg.list_keys()[0]['fingerprint']
            gpg.trust_keys(gpg.list_keys()[0]['fingerprint'], 'TRUST_FULLY')
            if DEBUG:
                print('Index \n', num, '\n')
                print('key list \n', gpg.list_keys()[0], '\n')
                print('S_fp \n', gpg.list_keys()[0]['fingerprint'], '\n')


Index 
 0 

key list 
 {'type': 'pub', 'trust': '-', 'length': '2048', 'algo': '1', 'keyid': 'ADACF6B6D1D9E513', 'date': '1526563261', 'expires': '', 'dummy': '', 'ownertrust': 'f', 'sig': '', 'cap': 'scESC', 'issuer': '', 'flag': '', 'token': '', 'hash': '', 'curve': '', 'compliance': '23', 'updated': '', 'origin': '0', 'uids': ['CPGP_GXS_2048'], 'sigs': [], 'subkeys': [['6E816E8CB90E1E4D', 'e', '67B65FAC2CB8DF98D12DCF496E816E8CB90E1E4D'], ['E0976422F4D47EA2', 'e', '6DE85095478FF5C52709095DE0976422F4D47EA2'], ['98543CCD0B65CBA6', 'e', 'EA9B49F22517F06F15A62EE898543CCD0B65CBA6']], 'fingerprint': '3A0AFF7D0095B25CE20A23CDADACF6B6D1D9E513', 'subkey_info': {'6E816E8CB90E1E4D': {'type': 'sub', 'trust': 'e', 'length': '2048', 'algo': '1', 'keyid': '6E816E8CB90E1E4D', 'date': '1526563261', 'expires': '1619776800', 'dummy': '', 'ownertrust': '', 'uid': '', 'sig': '', 'cap': 'e', 'issuer': '', 'flag': '', 'token': '', 'hash': '', 'curve': '', 'compliance': '23', 'updated': '', 'origin': 'unava

In [ ]:
# Generate self keys with export
input_data = gpg.gen_key_input(key_type=KeyType, key_length=KeyLen, name_real=UserKey, name_comment=Comment, name_email=EmailKey, passphrase=KeyPhrase)

# Key
key = gpg.gen_key(input_data)
if DEBUG: print("Key :\n", key)

# Fingerprint
S_fp = key.fingerprint
if DEBUG: print("Fingerprint :\n", S_fp)

# Clear Folder
shutil.rmtree(exppath)
os.makedirs(exppath)

# Export Public Key
ascii_armored_public_keys = gpg.export_keys(S_fp)
with open(pubkey, 'w') as f:
    f.write(ascii_armored_public_keys)

# Export Private Key
ascii_armored_private_keys = gpg.export_keys(S_fp, True)
with open(pvtkey, 'w') as f:
    f.write(ascii_armored_private_keys)


In [6]:
# Encrytion Program
filename = str(datetime.now().strftime("%Y%m%d%H%M%S%f")) + '.gpg'
with open('C:\\Users\\prashant\\Desktop\\PGP\\test.txt', 'r') as file:
    data = file.read()

if C_fp:
    status = gpg.encrypt(data, recipients=C_fp[0], sign=S_fp, passphrase=KeyPhrase,armor=False, always_trust=True, output=filename)
    if DEBUG: print("ok: ", status.ok, "\nstatus: ", status.status, "\nstderr: ", status.stderr)
else:
    print("error")
    log.update({'public Key': 'No Public Key Found. Please import keys...'})


ok:  False 
status:  error - sign-encrypt 17 
stderr:  gpg: Warning: not using '3A0AFF7D0095B25CE20A23CDADACF6B6D1D9E513' as default key: No secret key
gpg: all values passed to '--default-key' ignored
[GNUPG:] KEY_CONSIDERED 3A0AFF7D0095B25CE20A23CDADACF6B6D1D9E513 0
[GNUPG:] KEY_CONSIDERED 659A0D09BBB03399958AFD55FEF4724B0C7F50E1 0
gpg: no default secret key: No secret key
[GNUPG:] INV_SGNR 9
[GNUPG:] FAILURE sign-encrypt 17
gpg: [stdin]: sign+encrypt failed: No secret key

